# Building RAG from Scratch (Open-source only!)


- Sentence Transformers as the embedding model
- Postgres as the vector store (we support many other [vector stores](https://gpt-index.readthedocs.io/en/stable/module_guides/storing/vector_stores.html) too!)
- Llama 2 as the LLM (through [llama.cpp](https://github.com/ggerganov/llama.cpp))

## Setup

We setup our open-source components.
1. Sentence Transformers
2. Llama 2
3. We initialize postgres and wrap it with our wrappers/abstractions.

#### Sentence Transformers

In [1]:
#%pip install llama-index-readers-file pymupdf
#%pip install llama-index-vector-stores-postgres
#%pip install llama-index-embeddings-huggingface
#%pip install llama-index-llms-llama-cpp

In [2]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

/Users/busraoguzoglu/Desktop/Okul/586Project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Llama CPP

In this notebook, we use the [`llama-2-chat-13b-ggml`](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML) model, along with the proper prompt formatting.

Check out our [Llama CPP guide](https://gpt-index.readthedocs.io/en/stable/examples/llm/llama_2_llama_cpp.html) for full setup instructions/details.

In [3]:
#!pip install llama-cpp-python

In [4]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

/Users/busraoguzoglu/Desktop/Okul/586Project/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/busraoguzoglu/Desktop/Okul/586Project/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/busraoguzoglu/Desktop/Okul/586Project/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
llama_model_loader: loaded meta data

#### Initialize Postgres

Using an existing postgres running at localhost, create the database we'll be using.

**NOTE**: Of course there are plenty of other open-source/self-hosted databases you can use! e.g. Chroma, Qdrant, Weaviate, and many more. Take a look at our [vector store guide](https://gpt-index.readthedocs.io/en/stable/module_guides/storing/vector_stores.html).

**NOTE**: You will need to setup postgres on your local system. Here's an example of how to set it up on OSX: https://www.sqlshack.com/setting-up-a-postgresql-database-on-mac/.

**NOTE**: You will also need to install pgvector (https://github.com/pgvector/pgvector).

You can add a role like the following:
```
CREATE ROLE <user> WITH LOGIN PASSWORD '<password>';
ALTER ROLE <user> SUPERUSER;
```

In [5]:
#!pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

In [6]:
import psycopg2

# Define your database connection parameters
db_name = "rag_db"  # Use the existing database name here
host = "localhost"
password = "password"  # Replace with your actual PostgreSQL password
port = "5432"  # Default PostgreSQL port
user = "myuser"  # Replace with your actual PostgreSQL username

# Connect directly to the 'rag_db' database
conn = psycopg2.connect(
    dbname=db_name,  # Connect directly to 'rag_db'
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

# Optionally, perform any operations on 'rag_db' using a cursor
with conn.cursor() as c:
    # Drop and create operations are not needed if the database already exists
    # Here you can perform other database setup actions if necessary
    print(f"Connected to {db_name} successfully.")

# Close the connection when done
#conn.close()

Connected to rag_db successfully.


Create table (checks if exist, truncates if already exist and generates again)

In [7]:

from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError
from llama_index.vector_stores.postgres import PGVectorStore

# Connection parameters
db_params = {
    "user": "myuser",      # Replace with your actual username
    "password": "password",  # Replace with your actual password
    "host": "localhost",   # Adjust if your database is hosted elsewhere
    "port": "5432",        # Default PostgreSQL port
    "database": "rag_db"   # Replace with your actual database name
}

# Create SQLAlchemy engine
engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

table_name = "pmc_table"  # Your new table
embed_dim = 384  # Embedding dimension

try:
    with engine.connect() as conn:
        # Check if the table exists
        result = conn.execute(text(f"""
            SELECT EXISTS (
                SELECT FROM information_schema.tables 
                WHERE table_schema = 'public' AND table_name = :table_name
            );
        """), {"table_name": table_name}).scalar()

        if result:
            print(f"Table '{table_name}' exists. Cleaning it...")
            conn.execute(text(f"TRUNCATE TABLE {table_name};"))
        else:
            print(f"Table '{table_name}' does not exist. Creating it...")

        # Create a new table for vector store
        vector_store = PGVectorStore.from_params(
            database=db_params["database"],
            host=db_params["host"],
            password=db_params["password"],
            port=db_params["port"],
            user=db_params["user"],
            table_name=table_name,
            embed_dim=embed_dim,
        )
        print(f"Table '{table_name}' has been created.")
except OperationalError as e:
    print(f"Error: {e}")
    print("Make sure the database exists and connection parameters are correct.")


Table 'pmc_table' does not exist. Creating it...
Table 'pmc_table' has been created.


## Build an Ingestion Pipeline from Scratch

We show how to build an ingestion pipeline as mentioned in the introduction.

We fast-track the steps here (can skip metadata extraction). More details can be found [in our dedicated ingestion guide](https://gpt-index.readthedocs.io/en/latest/examples/low_level/ingestion.html).

## Load Data: Data is in 'data' folder

In [8]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.node_parser import SentenceSplitter

In [9]:
import re
# Define stop keywords
stop_keywords = ["References", "REFERENCES", "references", "Conflict of interest", "Conflicts of Interest", "Conflicts"]

def preprocess_text(text, stop_keywords):
    """Preprocess text by stopping at the earliest occurrence of any keyword, ignoring case."""
    earliest_position = len(text)  # Default to the end of the text
    
    for keyword in stop_keywords:
        pattern = rf"\b{re.escape(keyword)}\b"  # Match exact word boundaries
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            print(f"Keyword '{keyword}' found at position {match.start()}")  # Debugging
            earliest_position = min(earliest_position, match.start())
    
    # Truncate the text at the earliest keyword match
    return text[:earliest_position]

def clean_text(block_text):
    """Clean and process individual blocks of text."""
    block_text = " ".join(block_text.split())  # Remove extra spaces and newlines
    block_text = re.sub(r'^\d+\.\s+', '', block_text)  # Remove section numbering
    block_text = re.sub(r'[^A-Za-z0-9\s,.]', '', block_text)  # Keep only basic punctuation and letters
    
    if len(block_text) < 10 or re.match(r'Page \d+', block_text):
        return None  # Skip short blocks or page numbers
    return block_text

In [10]:
# Load and preprocess documents
folder_path = Path("./data")
documents = []

output_path = Path("./processed_documents.txt")  # File to save processed documents
with output_path.open("w", encoding="utf-8") as output_file:
    for pdf_file in folder_path.glob("*.pdf"):
        loader = PyMuPDFReader()
        loaded_docs = loader.load(file_path=str(pdf_file))
        print(f"{pdf_file.name}: {len(loaded_docs)} documents loaded")

        for doc_idx, doc in enumerate(loaded_docs):
            # Preprocess text
            processed_text = preprocess_text(doc.text, stop_keywords)
            
            # Clean and process each block of text
            cleaned_blocks = []
            for block in processed_text.split("\n"):
                cleaned_block = clean_text(block)
                if cleaned_block:
                    cleaned_blocks.append(cleaned_block)
            
            final_text = "\n".join(cleaned_blocks)
            doc.text = final_text  # Replace with cleaned and processed text
            doc.metadata = {"source": pdf_file.name}  # Add metadata
            documents.append(doc)
            
            # Write the processed text into the output file
            output_file.write(f"Document {doc_idx} from {pdf_file.name}:\n")
            output_file.write(final_text + "\n")
            output_file.write("="*80 + "\n")  # Separator between documents

print(f"Processed documents written to {output_path}")

5.pdf: 10 documents loaded
Keyword 'References' found at position 2699
Keyword 'REFERENCES' found at position 2699
Keyword 'references' found at position 2699
4.pdf: 12 documents loaded
Keyword 'References' found at position 1735
Keyword 'REFERENCES' found at position 1735
Keyword 'references' found at position 1735
1.pdf: 18 documents loaded
Keyword 'References' found at position 2673
Keyword 'REFERENCES' found at position 2673
Keyword 'references' found at position 2673
Keyword 'Conflicts of Interest' found at position 2604
Keyword 'Conflicts' found at position 2604
3.pdf: 16 documents loaded
Keyword 'References' found at position 1400
Keyword 'REFERENCES' found at position 1400
Keyword 'references' found at position 1400
2.pdf: 10 documents loaded
Keyword 'References' found at position 206
Keyword 'REFERENCES' found at position 206
Keyword 'references' found at position 206
Processed documents written to processed_documents.txt


In [11]:
print(documents[65])
print(len(documents))

Doc ID: 7a541f7f-4c9d-46a4-a0e6-ad205081280c
Text: MENGI ELIK et al. Roininen, K., Tuorila, H., Zandstra, E., de
Graaf, C., Vehkalahti, K., Stubenitsky, K.,  Mela, D. J. 2001.
Differences in health and taste attitudes and reported behaviour among
Finnish, Dutch and British consumers A crossnational validation of the
health and taste atti tude scales HTAS. Appetite, 371, 3345. Steptoe,
A., Wardle...
66


### 2. Use a Text Splitter to Split Documents

In [12]:
from llama_index.core.node_parser import SentenceSplitter

text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [13]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    #print(f"Document {doc_idx} has {len(cur_text_chunks)} chunks")
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))  

In [14]:
print(len(text_chunks))

94


### 3. Manually Construct Nodes from Text Chunks

In [15]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
        metadata={
            **documents[doc_idxs[idx]].metadata,  # This now includes 'source'
            "doc_index": doc_idxs[idx]  # Optional: document index
        }
    )
    nodes.append(node)

In [16]:
nodes[1]

TextNode(id_='954e2440-6b31-4311-ba2e-d56cb9a21226', embedding=None, metadata={'source': '5.pdf', 'doc_index': 1}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='This change in the lifestyle of the population is partly, due to\nthe need for measures to contain the spread of COVID19, such\nas social isolation. This has proven to be effective8 and has led\nmany people to perform their work remotely.9 This mobility\nrestriction has had direct effects on psychological factors, such\nas an increase in cases of anxiety and depression and a reduction\nin the practice of physical activities.1012 In addition, eating\nhabits were also inuenced both by economic factors, due to the\nreduction in the populations income, as well as by the\nconsumption of foods with higher energy density.13,14\nIn the period before the pandemic, the consumption of fresh\nand minimally processed foods represented approximately 70\nof the total caloric intake by the Brazilian po

### 4. Generate Embeddings for each Node

Here we generate embeddings for each Node using a sentence_transformers model.

In [17]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="text")
    )
    node.embedding = node_embedding

In [18]:
print(type(nodes[0].embedding))
print(nodes[0].embedding)
print(len(nodes))

<class 'list'>
[0.0003987177333328873, 0.02646005153656006, 0.03513289615511894, 0.007168531883507967, 0.041294097900390625, 0.0480182059109211, 0.023377425968647003, -0.004793648142367601, 0.00036878735409118235, -0.017007548362016678, -0.01259664911776781, -0.07230765372514725, 0.00043807117617689073, 0.019409792497754097, -0.006811037659645081, -0.024709824472665787, 0.0009681543451733887, -0.022223491221666336, 0.003708706935867667, 0.01637529581785202, -0.014613424427807331, -0.010371314361691475, -0.019866863265633583, -0.012201068922877312, 0.04210752621293068, 0.02097344398498535, -0.02411791682243347, -0.041234202682971954, -0.06391127407550812, -0.197813481092453, -0.025465227663517, -0.043725188821554184, 0.0314055010676384, -0.011092374101281166, -0.055188607424497604, -0.011988278478384018, 0.03101562149822712, 0.014969510026276112, -0.010766901075839996, -0.0149585772305727, -0.0023336694575846195, 0.029446586966514587, 0.03295325115323067, -0.040839437395334244, -0.02298

### 5. Load Nodes into a Vector Store

We now insert these nodes into our `PostgresVectorStore`.

In [19]:
vector_store.add(nodes)

['ed59b761-ad51-49e5-b441-531f74f300a6',
 '954e2440-6b31-4311-ba2e-d56cb9a21226',
 'c42e58d9-8245-48ac-a949-34ce9ab5b173',
 'ead0bc15-b22f-4c4d-97ae-684fb52e1533',
 '96e0f4d6-e732-417c-bb6c-5e789f4075d8',
 '8f73d683-6eee-41ff-93a6-df3d8b32b458',
 '7f899e6b-4b5c-4992-9411-e70b1cda018c',
 '21792127-cf8e-434c-a66b-4d80cd72b34c',
 '23e4bccf-181c-45b0-931a-808bcbb1d1d9',
 'cc13df31-66a9-4f3e-a83e-efc469ed7665',
 '49c83129-1914-4cc6-b4b4-2ca11fd5c500',
 '935a7e20-3dc4-4211-af52-6f600a2c6b34',
 '16981e20-f893-4489-93e9-171eb3d68fe7',
 'a2d4709c-1174-48f9-ae7e-49f9e0512ecd',
 'd208d433-27f4-4946-8679-568e75e95a98',
 '0dc6e1d0-65fe-4211-b930-d3cd33217d6e',
 '08d62893-e815-4a8d-a0fe-6ecb25ad4ff6',
 'da0c9361-46f8-4f40-aca2-338c6bc54854',
 '57b9b976-14d1-427e-9e8c-f70d8d6d4a1d',
 '91460084-cbee-4a31-ada7-ccaefd358378',
 '8462dce4-fd26-49bc-b1a6-9c499a49a291',
 '8be20ab4-5710-4709-a8c8-a39f1968fa31',
 'ddeba639-dbb8-494d-a65c-edba2734796a',
 '84a0a813-97eb-4bcf-9a1e-f76cfe5e8b7b',
 '5b3337fa-0fe5-

## Build Retrieval Pipeline from Scratch

We show how to build a retrieval pipeline. Similar to ingestion, we fast-track the steps. Take a look at our [retrieval guide](https://gpt-index.readthedocs.io/en/latest/examples/low_level/retrieval.html) for more details!

In [20]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List
from llama_index.core.schema import NodeWithScore
from typing import Optional
from llama_index.core.vector_stores import VectorStoreQuery

In [21]:
class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [22]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

retriever

## Plug this into our RetrieverQueryEngine to synthesize a response

In [23]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [24]:
query_str = "Which individuals play a central role in promoting healthy eating?"

response = query_engine.query(query_str)


llama_print_timings:        load time =    9961.08 ms
llama_print_timings:      sample time =       0.98 ms /    39 runs   (    0.03 ms per token, 39674.47 tokens per second)
llama_print_timings: prompt eval time =   24942.83 ms /  1188 tokens (   21.00 ms per token,    47.63 tokens per second)
llama_print_timings:        eval time =    4157.15 ms /    38 runs   (  109.40 ms per token,     9.14 tokens per second)
llama_print_timings:       total time =   29114.49 ms /  1226 tokens


In [25]:
print(str(response))

 Professionals in the fields of health and food, such as dietitians, food scientists and technologists, play a central role in promoting healthy eating.


In [26]:
print(response.source_nodes[0].get_content())

Nutrients 2024, 16, 1365
2 of 18
reaching 16% of the adult population in 2022 [4]. At the same time, nowadays, cardiovascular
diseases are the main cause of death globally, representing 32% of all global deaths [5]. Parallel
and paradoxically, thinness has become progressively valued and fatphobia, the discrimina-
tion and stigmatization against fat individuals, has become a phenomenon widely present
worldwide [6–8]. Finally, food crises with important health, social and economic international
impacts, such as the “mad cow”, have also increased concerns about food consumption [9].
Attempts to define what constitutes a healthy diet and provide dietary recommenda-
tions have been made in different scientific fields and organizations, including the World
Health Organization (WHO). In response to the increase in prevalence rates of morbidity
and mortality associated with chronic non-communicable diseases, in 2004 the WHO ap-
proved the Global Strategy on Diet, Physical Activity and Health,

In [27]:
query_str = "What does nutrition literacy significantly predicted in the social-ecological framework,?"

response = query_engine.query(query_str)

Llama.generate: 13 prefix-match hit, remaining 505 prompt tokens to eval

llama_print_timings:        load time =    9961.08 ms
llama_print_timings:      sample time =       1.22 ms /    48 runs   (    0.03 ms per token, 39312.04 tokens per second)
llama_print_timings: prompt eval time =    9977.23 ms /   505 tokens (   19.76 ms per token,    50.62 tokens per second)
llama_print_timings:        eval time =    5395.41 ms /    47 runs   (  114.80 ms per token,     8.71 tokens per second)
llama_print_timings:       total time =   15390.01 ms /   552 tokens


In [28]:
print(str(response))

 Based on the given context information, nutrition literacy significantly predicted in the social-ecological framework is "food and nutrition literacy (FNLIT) and its predictors in primary schoolchildren in Iran."


In [29]:
response.source_nodes[0]

NodeWithScore(node=TextNode(id_='de7da549-082e-49ea-8ce1-da982efaba78', embedding=None, metadata={'source': '4.pdf', 'doc_index': 20}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='; Omidvar, N.; Amini, M.; Abdollahi, M.; Eini-Zinab, H.; Amirhamidi, Z.;\nEsfandiari, S.; Nutbeam, D. Food and nutrition literacy (FNLIT) and its predictors in primary schoolchildren in Iran. Health\nPromot. Int. 2019, 34, 1002–1013. [CrossRef]\n40.\nLynn, M.R. Determination and quantiﬁcation of content validity. Nurs. Res. 1986, 35, 382–385. [CrossRef]\n41.\nNielsen-Bohlman, L.; Panzer, A.; Kindig, D. Health Literacy: A Prescription to End Confusion; National Academies Press: Washington,\nDC, USA, 2004.\n42.\nLiao, L.-L.; Lai, I.-J.; Shih, S.-F.; Chang, L.-C. Development and validation of the nutrition literacy measure for Taiwanese college\nstudents. Taiwan J. Public Health 2018, 37, 582–597.\n43.\nStory, M.; Neumark-Sztainer, D.; French, S. Individual and Environm

In [30]:
print(response.source_nodes[0].get_content())

; Omidvar, N.; Amini, M.; Abdollahi, M.; Eini-Zinab, H.; Amirhamidi, Z.;
Esfandiari, S.; Nutbeam, D. Food and nutrition literacy (FNLIT) and its predictors in primary schoolchildren in Iran. Health
Promot. Int. 2019, 34, 1002–1013. [CrossRef]
40.
Lynn, M.R. Determination and quantiﬁcation of content validity. Nurs. Res. 1986, 35, 382–385. [CrossRef]
41.
Nielsen-Bohlman, L.; Panzer, A.; Kindig, D. Health Literacy: A Prescription to End Confusion; National Academies Press: Washington,
DC, USA, 2004.
42.
Liao, L.-L.; Lai, I.-J.; Shih, S.-F.; Chang, L.-C. Development and validation of the nutrition literacy measure for Taiwanese college
students. Taiwan J. Public Health 2018, 37, 582–597.
43.
Story, M.; Neumark-Sztainer, D.; French, S. Individual and Environmental Inﬂuences on Adolescent Eating Behaviors. J. Am. Diet.
Assoc. 2002, 102, S40–S51. [CrossRef]
44.
Tabbakh, T.; Freeland-Graves, J. Development and validation of the Multidimensional Home Environment Scale (MHES) for
adolescents an